In [ ]:
import os
import h5py
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from time import strftime
from scipy.signal import spectrogram, stft, istft

In [ ]:
MODEL_NAME = "GanPlayground"

In [ ]:
def plot_all(do, label, fs=100, nperseg=150):
    d0 = pd.DataFrame(data=do[0])
    d1 = pd.DataFrame(data=do[1])
    d2 = pd.DataFrame(data=do[2])

    fig = plt.figure(figsize=(16, 10), dpi=80)
    ax1 = plt.subplot2grid((5, 6), (0, 0), colspan=3)
    ax2 = plt.subplot2grid((5, 6), (1, 0), colspan=3)
    ax3 = plt.subplot2grid((5, 6), (2, 0), colspan=3)
    ax4 = plt.subplot2grid((5, 6), (0, 3), colspan=3)
    ax5 = plt.subplot2grid((5, 6), (1, 3), colspan=3)
    ax6 = plt.subplot2grid((5, 6), (2, 3), colspan=3)
    ax7 = plt.subplot2grid((5, 6), (3, 0), colspan=2, rowspan=2)
    ax8 = plt.subplot2grid((5, 6), (3, 2), colspan=2, rowspan=2)
    ax9 = plt.subplot2grid((5, 6), (3, 4), colspan=2, rowspan=2)
    
    plt.subplots_adjust(hspace=1, wspace=1)

    sns.lineplot(data=d0, ax=ax1, linewidth=1, legend=None)
    sns.lineplot(data=d1, ax=ax2, linewidth=1, legend=None)
    sns.lineplot(data=d2, ax=ax3, linewidth=1, legend=None)

    ax1.set_title("Vertical component waveform")
    ax1.set(xlabel="Samples", ylabel="Amplitude counts")
    ax1.locator_params(nbins=6, axis="y")

    ax2.set_title("North component waveform")
    ax2.set(xlabel="Samples", ylabel="Amplitude counts")
    ax2.locator_params(nbins=6, axis="y")

    ax3.set_title("East component waveform")
    ax3.set(xlabel="Samples", ylabel="Amplitude counts")
    ax3.locator_params(nbins=6, axis="y")
    
    f_0, t_0, Sxx_0 = spectrogram(x=do[0], fs=fs)
    f_1, t_1, Sxx_1 = spectrogram(x=do[1], fs=fs)
    f_2, t_2, Sxx_2 = spectrogram(x=do[2], fs=fs)

    ax4.clear()
    ax4.set_title("Vertical component spectrogram")
    ax4.pcolormesh(t_0, f_0, Sxx_0, shading="gouraud")
    ax4.set(xlabel="Time [sec]", ylabel="Frequency [Hz]")

    ax5.clear()
    ax5.set_title("North component spectrogram")
    ax5.pcolormesh(t_1, f_1, Sxx_1, shading="gouraud")
    ax5.set(xlabel="Time [sec]", ylabel="Frequency [Hz]")

    ax6.clear()
    ax6.set_title("East component spectrogram")
    ax6.pcolormesh(t_2, f_2, Sxx_2, shading="gouraud")
    ax6.set(xlabel="Time [sec]", ylabel="Frequency [Hz]")
    
    f_sftt_0, t_sftt_0, Zxx_0 = stft(do[0], window='hanning', fs=fs, nperseg=nperseg)
    f_sftt_1, t_sftt_1, Zxx_1 = stft(do[1], window='hanning', fs=fs, nperseg=nperseg)
    f_sftt_2, t_sftt_2, Zxx_2 = stft(do[2], window='hanning', fs=fs, nperseg=nperseg)
    
    ax7.clear()
    ax7.set_title("Vertical component STFT")
    ax7.pcolormesh(t_sftt_0, f_sftt_0, np.abs(Zxx_0), shading='auto')
    
    ax8.clear()
    ax8.set_title("North component STFT")
    ax8.pcolormesh(t_sftt_1, f_sftt_1, np.abs(Zxx_1), shading='auto')
    
    ax9.clear()
    ax9.set_title("East component STFT")
    ax9.pcolormesh(t_sftt_2, f_sftt_2, np.abs(Zxx_2), shading='auto')

    plt.suptitle(label, fontsize=14)

In [ ]:
def plot_stft(stream, fs=100, nperseg=155):
    f, t, Zxx = stft(stream, window='hanning', fs=fs, nperseg=nperseg)
    # plt.specgram(x_1[0][0], cmap='plasma', Fs=100)
    plt.pcolormesh(t, f, np.abs(Zxx), shading='auto')

In [ ]:
def get_data(file_path, idx_start, idx_end, idx_slice):
    x_train = None
#     y_train = None
    evi_train = None
    x_test = None
#     y_test = None
    evi_test = None
    with h5py.File(file_path, "r") as f:
        x_train = f["data"][idx_start:idx_slice]
#         y_train = f["labels"][idx_start:idx_slice]
        evi_train = f["keys"][idx_start:idx_slice]
        x_test = f["data"][idx_slice:idx_end]
#         y_test = f["labels"][idx_slice:idx_end]
        evi_test = f["keys"][idx_slice:idx_end]
#         return (x_train, y_train, evi_train, x_test, y_test, evi_test)
        return (x_train, evi_train, x_test, evi_test)

In [ ]:
def build_stfts(x):
    x_train = []

    for idx, triplet in enumerate(x):
        for stream in triplet:
            _, _, zxx = stft(stream, window='hanning', nperseg=155)
            x_train.append(np.abs(zxx))

    return (x_train)

# Read processed data

`x_?` is a dataset of streams:

`x_?[0][0]` -> Z component

`x_?[0][1]` -> N component

`x_?[0][2]` -> E component

`y_?` is a dataset of labels corresponding to streams. `1` means earthquake, `0` means noise.

In [ ]:
# (x_1, y_1, evi_1, x_2, y_2, evi_2) = get_data("../data/STEAD-processed-gan.hdf5", 10000, 20000, 18000)
(x_1, evi_1, x_2, evi_2) = get_data("../data/STEAD-processed-gan.hdf5", 10000, 20000, 18000)

In [ ]:
idx = 12
plot_all(x_1[idx], evi_1[idx].decode("utf-8"))

# Convert streams to STFT

In [ ]:
# STFT of the stream and then reverse STFT back into original stream
# f, t, Zxx = stft(x_1[1][0], window='hanning', fs=100, nperseg=155)
# k2 = istft(Zxx, window='hanning', fs=100, nperseg=155)

In [ ]:
%%time
x_train = build_stfts(x_1)
x_test = build_stfts(x_2)

# Reshape data

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 1620)
x_test = x_test.reshape(x_test.shape[0], 1620)

In [ ]:
X = np.expand_dims(x_train, axis = 2)
X.shape

In [ ]:
Y = np.array(y_train)
Y.shape

In [ ]:
T = np.expand_dims(x_test, axis = 2)
T.shape

In [ ]:
Q = np.array(y_test)
Q.shape

# Logs and Tensorboard

In [ ]:
folder_name = f"{MODEL_NAME} at {strftime('%H:%M')}"
log_dir = os.path.join("../log/", folder_name)

try:
    os.makedirs(log_dir)
except OSError as exception:
    print(exception.strerror)
else:
    print("Successfully created dirs!")

# Define GAN

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(3*3*30, use_bias=False, input_shape=(78,78)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Reshape((3, 3, 30)))

    model.add(tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(1, 9), padding='same', use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(1, 5), padding='same', use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Conv2DTranspose(1, (5, 5), strides=(1, 4), padding='same', use_bias=False, activation='tanh'))

    return model

In [ ]:
generator = make_generator_model()

noise = tf.random.normal(dtype=tf.dtypes.float32, shape=[78, 78], stddev=5)
generated_image = generator(noise, training=False)

# TensorShape([1, 3, 540, 1])
generated_image.shape

In [ ]:
plot_all(generated_image[0, :, :, 0], "GAN Generator Noise")

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=adam,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
model.fit(X, Y, epochs=25, validation_data=(T, Q), callbacks=[tensorboard_callback])

In [ ]:
test_loss, test_acc = model.evaluate(T, Q, verbose=2)